Functional Programming

In [5]:
lapply(mtcars, function(x) mean(x)/sd(x))      

In [8]:
integrate(function(x) x^2-x, 0, 10)
integrate(function(x) sin(x)+cos(x), -pi, pi)
integrate(function(x) exp(x)/x, 10, 20)

283.3333 with absolute error < 3.1e-12

2.615901e-16 with absolute error < 6.3e-14

25613160 with absolute error < 2.8e-07

In [10]:
power <- function(exponent){
    function(x) x^exponent
}
square <- power(2)
cube <- power(3)
as.list(environment(square))
as.list(environment(cube))

$exponent
[1] 2

$exponent
[1] 3

In [14]:
library(pryr)
unenclose(square)
unenclose(cube)

function (x) 
x^2

function (x) 
x^3

In [19]:
power <- function(exponent) { 
    print(environment()) 
    function(x) x ^ exponent
}
zero <- power(0) 
environment(zero)

<environment: 0x000000001401af40>


<environment: 0x000000001401af40>

In [16]:
i <- 0
new_counter2 <- function() { 
    i <<- i + 1 
    i
}
new_counter2()

[1] 1

function() { 
        i <<- i + 1
        i
    }
<environment: 0x0000000005f0eeb0>

In [44]:
new_counter3 <- function() { 
    i <- 0
    function() { 
        i <- i + 1
        i
    } 
}
new_counter <- new_counter3()
new_counter()

[1] 1

#### *Exercise 1* 
- Create a function that creates functions that compute the ith central mo- ment4 of a numeric vector and then test it with `stopifnot()`.

In [74]:
x <- runif(100)
cntr_mmt <- function(x){
    mmt <- function(k){
        if (k==1){
            mean(x)
        } else{
            mean((x-mean(x))^k)
        }       
    }
}
moment <- cntr_mmt(x)
stopifnot(all.equal(moment(2), var(x)*99/100))
stopifnot(all.equal(moment(1), mean(x)))

#### *Exercise 2* 
- Create a function `pick()`, that takes an index, i, as an argument and returns a function an argument x that subsets x with i.

In [75]:
pick <- function(i){
    function(x) x[[i]]
}
lapply(mtcars, pick(5))

$mpg
[1] 18.7

$cyl
[1] 8

$disp
[1] 360

$hp
[1] 175

$drat
[1] 3.15

$wt
[1] 3.44

$qsec
[1] 17.02

$vs
[1] 0

$am
[1] 0

$gear
[1] 3

$carb
[1] 2

- Functions also can be used in a list, and then use `lapply(funs, function(f) f())` to execute.

> funs <- list(
    sum = sum,
    mean = mean,
    median = median
)

> lapply(funs, function(f) f(1:10))

- If we want to remove `NA` for every function in list, we can write a closure to abstract.

> lapply(funs, function(f) f(x, na.rm = TRUE))

#### *Exercise 3* 
- Implement a summary function that works like `base::summary()`, but takes a list of functions to use to compute the summary. Modify the function so it returns a closure, making it possible to use it as a function factory.

#### *Exercise 4* 
- Create a named list of all base functions. Use `ls()`, `get()` and `is.function()`. Use that list of functions to answer the following questions:
– Which base function has the most arguments? 
– How many base functions have no arguments?

In [18]:
library(data.table)
named.list <- as.data.table(ls(baseenv()))
named.list <- named.list[, V2 := lapply(V1, get)
    ][, V3 := unlist(lapply(V2, is.function))
    ][V3 == TRUE
    ][, V4 := lapply(V2, formalArgs)]
named.list[, V5 := lapply(V4, length), by = .(V1)
    ][, max(V5)]
named.list[V5==0, .N]

[1] 22

[1] 224

#### *Exercise 5*
Which of the following commands is with(x, f(z)) equivalent to?

**(a)** x$f(x$z)

**(b)** f(x$z)

**(c)** x$f(z)

**(d)** f(z)

Answer : **(c)**

#### *Exercise 6* 
Instead of creating individual fuctions `midpoint()`, `trapezoid()`, `simpson()` etc, we could store them in a list. If we do that, how does the code change? Can you create the list of functions from a list of coefficients for the *Newton-Cotes* formulae?

In [ ]:
newton_cotes <- function(coef, open = FALSE) { 
    n <- length(coef) + open
    function(f, a, b) {
        pos <- function(i) a + i * (b - a) / n 
        points <- pos(seq.int(0, length(coef) - 1))
        (b - a) / sum(coef) * sum(f(points) * coef)
    } 
} 

collection_cotes<- list(
    midpoint = newton_cotes(1, open = TURE)
    trapezoid = newton_cotes(c(1, 1))
    simpson = nweton_cotes(c(1, 4, 1))    
)

#### *Exercise 7*
The tradeoff in integration rules is that more complex rules are slower to compute, but need fewer pieces. For `sin()` in the range [0, pi], determine the number of pieces needed to for each rule to be equally accurate. Illustrate your results with a graph. How do they change for different functions? `sin(1/x^2)` is particularly challenging.

#### *Exercise 8*
For each of the *Newton-Cotes* rules, how many pieces do you need to get within 0.1% of the true answer for `sin()` in the range [0, pi]. Write a function that determines that automatically for any function (hint: look at `optim()` and construct a one-argument function with closures)